<a href="https://colab.research.google.com/github/lakshhk4/SquadShifter/blob/main/FrepData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = '/content/footballData.csv'
df = pd.read_csv(path)
print(df.shape)
df.head()


(12149, 175)


,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,Team Country,Player Lower,First Name Lower,Last Name Lower,First Initial Lower,Team Country Lower,Nationality Code,Nationality Cleaned,Position Grouped,Outfielder Goalkeeper
0,Aaron Cresswell,eng ENG,DF,West Ham,eng Premier League,27,1989.0,36,35,3069,...,England,aaron cresswell,aaron,cresswell,a,england,ENG,England,Defender,Outfielder
1,Aaron Hunt,de GER,"MF,FW",Hamburger SV,de Bundesliga,30,1986.0,28,26,2081,...,Germany,aaron hunt,aaron,hunt,a,germany,GER,Germany,Midfielder,Outfielder
2,Aaron Lennon,eng ENG,MF,Burnley,eng Premier League,30,1987.0,14,13,1118,...,England,aaron lennon,aaron,lennon,a,england,ENG,England,Midfielder,Outfielder
3,Aaron Lennon,eng ENG,"FW,MF",Everton,eng Premier League,30,1987.0,15,9,793,...,England,aaron lennon,aaron,lennon,a,england,ENG,England,Forward,Outfielder
4,Aaron Mooy,au AUS,MF,Huddersfield,eng Premier League,26,1990.0,36,34,3067,...,England,aaron mooy,aaron,mooy,a,england,AUS,Australia,Midfielder,Outfielder


In [ ]:
column_info = df.dtypes.reset_index()
column_info.columns = ['Column Name', 'Data Type']

# Get the list of column names with data types
column_list = column_info.values.tolist()
for i in column_list:
  print(i)

#df.columns.values.tolist()

['Player', dtype('O')]
['Nation', dtype('O')]
['Pos', dtype('O')]
['Squad', dtype('O')]
['Comp', dtype('O')]
['Age', dtype('int64')]
['Born', dtype('float64')]
['MP', dtype('int64')]
['Starts', dtype('int64')]
['Min', dtype('int64')]
['90s', dtype('float64')]
['Gls', dtype('int64')]
['Ast', dtype('int64')]
['G-PK', dtype('int64')]
['PK', dtype('int64')]
['PKatt', dtype('int64')]
['CrdY', dtype('int64')]
['CrdR', dtype('int64')]
['Gls.1', dtype('float64')]
['Ast.1', dtype('float64')]
['G+A', dtype('float64')]
['G-PK.1', dtype('float64')]
['G+A-PK', dtype('float64')]
['xG', dtype('float64')]
['npxG', dtype('float64')]
['xA', dtype('float64')]
['npxG+xA', dtype('float64')]
['xG.1', dtype('float64')]
['xA.1', dtype('float64')]
['xG+xA', dtype('float64')]
['npxG.1', dtype('float64')]
['npxG+xA.1', dtype('float64')]
['Matches', dtype('O')]
['Sh', dtype('float64')]
['SoT', dtype('int64')]
['SoT%', dtype('float64')]
['Sh/90', dtype('float64')]
['SoT/90', dtype('float64')]
['G/Sh', dtype('float

Age to Won% --> use ints

In [ ]:
df['Season'].unique()

array(['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022',
       nan], dtype=object)

In [ ]:
print(df["Position Grouped"].unique())
print(df["Team Country"].unique())
print(df["Comp"].unique())

['Defender' 'Midfielder' 'Forward' 'Goalkeeper' nan]
['England' 'Germany' 'Spain' 'France' 'Italy' nan]
['eng Premier League' 'de Bundesliga' 'es La Liga' 'fr Ligue 1'
 'it Serie A']


In [ ]:
len(df["Player"].unique().tolist()) # number of players

4813

'Gls',  'Ast' --> lets start with focusing only on forwards, see how goal and assists change with team around them


In [ ]:
df_2017 = df[df['Season'] == '2017-2018'].copy()
df_2018 = df[df['Season'] == '2018-2019'].copy()
print(df_2017.shape)
print(df_2018.shape)




(2687, 175)
(2658, 175)


In [ ]:
# Merge the two DataFrames based on the player's name
merged_df = pd.merge(df_2017, df_2018, on='Player', how='inner')
merged_df.columns.values.tolist()

# Filter the merged DataFrame to find players who switched teams
switched_players = merged_df[merged_df['Squad_x'] != merged_df['Squad_y']]

# Get the unique list of switched players
unique_switched_players = switched_players[['Player','Squad_x','Squad_y']].drop_duplicates()

# Print the list of unique switched players
print(switched_players.shape)
print(unique_switched_players)
print(unique_switched_players.shape)

(619, 349)
                   Player           Squad_x         Squad_y
2            Aaron Lennon           Everton         Burnley
6          AarÃ³n MartÃ­n          Espanyol        Mainz 05
9            Abdou Diallo          Mainz 05        Dortmund
14          Achraf Hakimi       Real Madrid        Dortmund
17            Adam Masina           Bologna         Watford
...                   ...               ...             ...
1982    Ãlvaro Odriozola     Real Sociedad     Real Madrid
1986  ÃaÄlar SÃ¶yÃ¼ncÃ¼          Freiburg  Leicester City
1997   Åukasz FabiaÅski      Swansea City        West Ham
1999    Åukasz Skorupski              Roma         Bologna
2000       Å ime Vrsaljko  AtlÃ©tico Madrid           Inter

[619 rows x 3 columns]
(619, 3)


In [ ]:
# can access switchedPlayers
def  aggregator(seasonData, teamOrLeague, suffixString, row, oldOrNew):
  code = "_x" if oldOrNew == "old" else "_y"

  if teamOrLeague == "team":
    tolName = row[1]["Squad"+code]
    queryPlayers = seasonData[seasonData['Squad'] == tolName].copy() #remove player in Q itself
    queryPlayers = queryPlayers[queryPlayers['Player'] != row[1]["Player"]] # could add this line in else, to remove player who was also on loan in that szn
  elif teamOrLeague == "league":
    tolName = row[1]["Comp"+code]
    queryPlayers = seasonData[seasonData['Comp'] == tolName].copy() #remove team from query.
    queryPlayers = queryPlayers[queryPlayers['Squad'] != row[1]["Squad"+code]]


  #pd.set_option('display.max_rows', None)
  #print(queryPlayers[['Player', 'Squad','Comp']]) #test whether intented group is queried
  oldPlayersATT = queryPlayers[queryPlayers['Position Grouped'] == "Forward"].copy()
  oldPlayersMID = queryPlayers[queryPlayers['Position Grouped'] == "Midfielder"].copy()
  oldPlayersDEF = queryPlayers[queryPlayers['Position Grouped'] == "Defender"].copy()
  oldPlayersGK = queryPlayers[queryPlayers['Position Grouped'] == "Goalkeeper"].copy()

  #AGGREGATION OLD ATTACKERS
  start_index = 5  # Starting column index (inclusive)
  end_index = 161  # Ending column index (exclusive)
  column_range = oldPlayersATT.iloc[:, start_index:end_index]

  column_mean = column_range.mean()
  old_forwards = pd.DataFrame(column_mean).transpose()
  old_forwards = old_forwards.add_suffix(suffixString+'_FWD')

   #AGGREGATE OLD MIDFIELD
  column_range = oldPlayersMID.iloc[:, start_index:end_index]
  #print("old mid size: " + str(column_range.shape))
  column_mean = column_range.mean()
  old_midfielders = pd.DataFrame(column_mean).transpose()
  old_midfielders = old_midfielders.add_suffix(suffixString+'_MID')
  #print(old_midfielders)

  #AGGREGATE OLD DEFENSE
  column_range = oldPlayersDEF.iloc[:, start_index:end_index]
  #print("old def size: " + str(column_range.shape))
  column_mean = column_range.mean()
  old_defenders = pd.DataFrame(column_mean).transpose()
  old_defenders = old_defenders.add_suffix(suffixString+'_DEF')
  #print(old_defenders)

  # COMBINING AGGREGATED DATA, WITH EXISTING ROW DATA
  combined_Old_df = pd.concat([old_forwards, old_midfielders, old_defenders], axis=1)
  return combined_Old_df

In [ ]:
def aggregateOld(row, dfOld, dfNew):
  combined_oldT_df = aggregator(dfOld, 'team', '_OldT', row, "old") #each adds 3 x 150 features
  combined_oldO_df = aggregator(dfOld, 'league', '_OldO', row, "old")

  combined_newT_df = aggregator(dfNew, 'team', '_NewT', row, "new")
  combined_newO_df = aggregator(dfNew, 'league', '_NewO', row, "new")

  row_df = pd.DataFrame(row[1]).transpose()

  row_df.reset_index(drop=True, inplace=True)
  combined_oldT_df.reset_index(drop=True, inplace=True)
  combined_oldO_df.reset_index(drop=True, inplace=True)
  combined_newT_df.reset_index(drop=True, inplace=True)
  combined_newO_df.reset_index(drop=True, inplace=True)
  #print(row_df)
  new_df_row = pd.concat([row_df,combined_oldT_df,combined_oldO_df,combined_newT_df,combined_newO_df], axis = 1, ignore_index=False)
  #print(new_df_row)
  return new_df_row


In [ ]:
new_df = pd.DataFrame()

def shiftedPlayers(df1, df2):
  merged_df = pd.merge(df1, df2, on='Player', how='inner')
  merged_df.columns.values.tolist()

  # Filter the merged DataFrame to find players who switched teams
  switched_players = merged_df[merged_df['Squad_x'] != merged_df['Squad_y']]

  # Get the unique list of switched players
  unique_switched_players = switched_players[['Player','Squad_x','Squad_y']].drop_duplicates()

  # Print the list of unique switched players
  #print(switched_players.shape)
  #print(unique_switched_players)
  #print(unique_switched_players.shape)
  return switched_players

#'2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022',
df_2019 = df[df['Season'] == '2019-2020'].copy()
df_2020 = df[df['Season'] == '2020-2021'].copy()
df_2021 = df[df['Season'] == '2021-2022'].copy()

switched_players17_18 = shiftedPlayers(df_2017, df_2018)
switched_players18_19 = shiftedPlayers(df_2018, df_2019)
switched_players19_20 = shiftedPlayers(df_2019, df_2020)
switched_players20_21 = shiftedPlayers(df_2020, df_2021)
print('---')
print(switched_players17_18.shape)
print(switched_players18_19.shape)
print(switched_players19_20.shape)
print(switched_players20_21.shape)


for row in switched_players17_18.iterrows():
  #print(row[1]["Player"])
  old_df = aggregateOld(row, df_2017, df_2018)
  new_df = new_df.append(old_df,ignore_index=True)

for row in switched_players18_19.iterrows():
  #print(row[1]["Player"])
  old_df = aggregateOld(row, df_2018, df_2019)
  new_df = new_df.append(old_df,ignore_index=True)

for row in switched_players19_20.iterrows():
  #print(row[1]["Player"])
  old_df = aggregateOld(row, df_2019, df_2020)
  new_df = new_df.append(old_df,ignore_index=True)

for row in switched_players20_21.iterrows():
  #print(row[1]["Player"])
  old_df = aggregateOld(row, df_2020, df_2021)
  new_df = new_df.append(old_df,ignore_index=True)









In [ ]:
print(new_df.shape)
#print(new_df.head())
#pd.reset_option('display.max_rows')
#print(new_df[["Player","Squad_x", "Squad_y","Comp_x", "WonOldO_DEF", "LostOldO_DEF", "Won%OldO_DEF"]])
df_rounded = new_df.round(3)
print(df_rounded.shape)
new_df.tail()

(1997, 2212)
(1997, 2212)


,Player,Nation_x,Pos_x,Squad_x,Comp_x,Age_x,Born_x,MP_x,Starts_x,Min_x,...,PKwon_NewO_DEF,PKcon_NewO_DEF,OG_NewO_DEF,Recov_NewO_DEF,Won_NewO_DEF,Lost_NewO_DEF,Won%_NewO_DEF,Matches_NewT_MID,Matches_NewT_DEF,Matches_NewT_FWD
1992,Memphis Depay,nl NED,FW,Lyon,fr Ligue 1,26,1994.0,37,33,2923,...,0.011765,0.094118,0.011765,13.717647,3.388235,2.447059,55.458108,NaN,NaN,NaN
1993,Merih Demiral,tr TUR,DF,Juventus,it Serie A,22,1998.0,15,10,940,...,0.012658,0.037975,0.000000,9.759494,2.012658,1.607595,54.355882,NaN,NaN,NaN
1994,Mike Maignan,fr FRA,GK,Lille,fr Ligue 1,24,1995.0,38,38,3420,...,0.012048,0.036145,0.000000,10.084337,2.265060,1.530120,55.294286,NaN,NaN,NaN
1995,Milot Rashica,xk KVX,"FW,MF",Werder Bremen,de Bundesliga,24,1996.0,24,14,1462,...,0.000000,0.077922,0.000000,16.597403,4.584416,3.012987,59.013889,NaN,NaN,NaN
1996,Mitchel Bakker,nl NED,DF,Paris S-G,fr Ligue 1,20,2000.0,26,18,1694,...,0.000000,0.026316,0.026316,15.394737,4.144737,2.763158,61.048529,NaN,NaN,NaN


In [ ]:
new_df.to_csv('transferredPlayersData.csv', index=True)

col_list = new_df.columns.tolist()
for i in col_list:
  print(i)

Player
Nation_x
Pos_x
Squad_x
Comp_x
Age_x
Born_x
MP_x
Starts_x
Min_x
90s_x
Gls_x
Ast_x
G-PK_x
PK_x
PKatt_x
CrdY_x
CrdR_x
Gls.1_x
Ast.1_x
G+A_x
G-PK.1_x
G+A-PK_x
xG_x
npxG_x
xA_x
npxG+xA_x
xG.1_x
xA.1_x
xG+xA_x
npxG.1_x
npxG+xA.1_x
Matches_x
Sh_x
SoT_x
SoT%_x
Sh/90_x
SoT/90_x
G/Sh_x
G/SoT_x
Dist_x
FK_x
npxG/Sh_x
G-xG_x
np:G-xG_x
Cmp_x
Att_x
Cmp%_x
TotDist_x
PrgDist_x
Cmp.1_x
Att.1_x
Cmp%.1_x
Cmp.2_x
Att.2_x
Cmp%.2_x
Cmp.3_x
Att.3_x
Cmp%.3_x
A-xA_x
KP_x
1/3_x
PPA_x
CrsPA_x
Prog_x
Live_x
Dead_x
TB_x
Press_x
Sw_x
Crs_x
CK_x
In_x
Out_x
Str_x
Ground_x
Low_x
High_x
Left_x
Right_x
Head_x
TI_x
Other_x
Off_x
Out.1_x
Int_x
Blocks_x
SCA_x
SCA90_x
PassLive_x
PassDead_x
Drib_x
Fld_x
Def_x
GCA_x
GCA90_x
PassLive.1_x
PassDead.1_x
Drib.1_x
Sh.1_x
Fld.1_x
Def.1_x
Tkl_x
TklW_x
Def 3rd_x
Mid 3rd_x
Att 3rd_x
Tkl.1_x
Tkl%_x
Past_x
Succ_x
%_x
Def 3rd.1_x
Mid 3rd.1_x
Att 3rd.1_x
ShSv_x
Pass_x
Tkl+Int_x
Clr_x
Err_x
Touches_x
Def Pen_x
Att Pen_x
Succ%_x
#Pl_x
Megs_x
Carries_x
CPA_x
Mis_x
Dis_x
Targ_x
Rec_x
Rec

In [ ]:
# 'Position Grouped'] == "Forward"
test = new_df[new_df['Position Grouped_x'] == "Forward"].copy()
pd.set_option('display.max_rows', None)
tester2 = test[["Player", "Squad_y","G+A_x","G+A_y"]]
#tester2['Percent Change'] = tester2['G+A_y'].pct_change() * 100
tester2['pct_change'] = tester2[['G+A_x', 'G+A_y']].pct_change(axis=1)['G+A_y'] * 100


#tester3 = test[["G+A_x","G+A_y"]]
tester2.replace([np.inf, -np.inf], np.nan, inplace=True)
tester2.replace([-100.000000, 100.000000], np.nan, inplace=True)
-100.000000


#"Gls_x","Gls_y"
print(tester2)
#print(tester3.pct_change())

print(tester2['pct_change'].mean())

                         Player           Squad_y  G+A_x  G+A_y   pct_change
0                  Aaron Lennon           Burnley   0.00   0.15          NaN
5                Adama Diakhaby      Huddersfield   0.42   0.00          NaN
14               Alassane PlÃ©a        M'Gladbach   0.59   0.58    -1.694915
20         Aleksandar MitroviÄ            Fulham   0.95   0.38   -60.000000
27            Alexander Esswein         Stuttgart   0.44   0.00          NaN
28              Alexandre Mendy          Guingamp   0.45   0.35   -22.222222
30              Alexis SÃ¡nchez    Manchester Utd   0.60   0.41   -31.666667
42               Andrea Petagna              SPAL   0.50   0.50     0.000000
43             Andrea Pinamonti         Frosinone   0.00   0.44          NaN
44            Andreas Cornelius          Bordeaux   0.39   0.45    15.384615
46            Andriy Yarmolenko          West Ham   0.52   0.39   -25.000000
48                  AndrÃ© Hahn          Augsburg   0.20   0.35    75.000000

<ipython-input-77-7fb03954c971>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tester2['pct_change'] = tester2[['G+A_x', 'G+A_y']].pct_change(axis=1)['G+A_y'] * 100
<ipython-input-77-7fb03954c971>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tester2.replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-77-7fb03954c971>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [ ]:
new_df['Season_y'].unique()

array(['2018-2019', '2019-2020', '2020-2021', '2021-2022'], dtype=object)

investigatwe